In [ ]:
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png' 
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)



In [ ]:

from braindecode.datasets.sensor_positions import get_channelpos, CHANNEL_10_20_APPROX



POS_APPROX = ('angle',
    ('A1', (-5.0, 0.0)),
    ('A2', (5.0, 0.0)),
    ('Fp1', (-3.5, 3.5)),
    ('Fp2', (3.5, 3.5)),
    ('F7', (-4.0, 2.0)),
    ('F3', (-2.0, 2.0)),
    ('Fz', (0.0, 2.0)),
    ('F4', (2.0, 2.0)),
    ('F8', (4.0, 2.0)),
    ('C3', (-2.0, 0.0)),
    ('Cz', (0.0, 0.0)),
    ('C4', (2.0, 0.0)),
    ('T3', (-3,0)),
    ('T4', (3,0)),
    ('P3', (-2.0, -2.0)),
    ('Pz', (0.0, -2.0)),
    ('P4', (2.0, -2.0)),
    ('T5', (-3.5,-2.)),
    ('T6', (3.5,-2.)),          
    ('O1', (-4, -3.5)),
    ('O2', (4, -3.5)))

ch_names = ['A1', 'A2', 'C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1',
                    'FP2', 'FZ', 'O1', 'O2',
                    'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
positions = [get_channelpos(name, POS_APPROX) for name in ch_names]
positions = np.array(positions)


### show many frequencies

In [ ]:
amp_pred_corrs, orig_acc, new_accs = np.load('data/models/pytorch/auto-diag/6000-input/21/gaussian.perturbation.30.npy')
mean_corr = np.mean(amp_pred_corrs, axis=0)

fs = 100.0
freqs = np.fft.rfftfreq(6000, d=1.0/fs)
assert freqs.shape[0] == mean_corr.shape[1]

In [ ]:
freq_start_stops = ((0,4), (4,8),(8,14),(14,20), (20,30),(30,50))

In [ ]:
from braindecode.visualization.plot import ax_scalp
from autodiag.plot import add_colorbar_to_scalp_grid
plt.plot(freqs, np.mean(mean_corr, axis=0)[:,1])
plt.xlabel('Freq [Hz]')
plt.ylabel('Average correlation with\n abnormal class', fontsize=14)
fig, axes = plt.subplots(1, len(freq_start_stops), figsize=(20,4))
for i_plot, (freq_start, freq_stop) in enumerate(freq_start_stops):
    i_start = np.searchsorted(freqs,freq_start)
    i_stop = np.searchsorted(freqs, freq_stop) + 1
    freq_corr = np.mean(mean_corr[:,i_start:i_stop], axis=1)
    max_abs_val = np.max(np.abs(freq_corr))
    ax = axes[i_plot]
    ax_scalp(freq_corr[:,1], ch_names, chan_pos_list=POS_APPROX, cmap=cm.coolwarm,
            vmin=-0.005, vmax=0.005, ax=ax,annotate=True)
    ax.set_title(u"{:d}—{:d} Hz".format(freq_start, freq_stop))
add_colorbar_to_scalp_grid(fig, axes, "Correlation", min_max_ticks=False)

In [ ]:
amp_pred_corrs, orig_acc, new_accs = np.load('data/models/pytorch/auto-diag/6000-input/22/gaussian.perturbation.30.npy')
mean_corr = np.mean(amp_pred_corrs, axis=0)

fs = 100.0
freqs = np.fft.rfftfreq(6000, d=1.0/fs)
assert freqs.shape[0] == mean_corr.shape[1]

print(open('data/models/pytorch/auto-diag/6000-input/22/config.json', 'r').read())


In [ ]:
from braindecode.visualization.plot import ax_scalp
plt.plot(freqs, np.mean(mean_corr, axis=0)[:,1])
plt.xlabel('Freq [Hz]')
plt.ylabel('Average correlation with\n abnormal class', fontsize=14)
fig, axes = plt.subplots(1, len(freq_start_stops), figsize=(20,4))
for i_plot, (freq_start, freq_stop) in enumerate(freq_start_stops):
    i_start = np.searchsorted(freqs,freq_start)
    i_stop = np.searchsorted(freqs, freq_stop) + 1
    freq_corr = np.mean(mean_corr[:,i_start:i_stop], axis=1)
    max_abs_val = np.max(np.abs(freq_corr))
    ax = axes[i_plot]
    ax_scalp(freq_corr[:,1], ch_names, chan_pos_list=POS_APPROX, cmap=cm.coolwarm,
            vmin=-0.005, vmax=0.005, ax=ax,annotate=True)
    ax.set_title(u"{:d}—{:d} Hz".format(freq_start, freq_stop))
add_colorbar_to_scalp_grid(fig, axes, "Correlation", min_max_ticks=False)

In [ ]:
np.max(np.abs(mean_corr))